In [8]:
import pandas as pd
import hvplot.pandas
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension()

In [16]:
# load the csv files into dataframes and do some simple cleaning
health_expenditure_df = pd.read_csv("health_expenditure_per_capita.csv")
health_expenditure_df.dropna(inplace=True)

covid_df = pd.read_csv("JHcovid12-31-2020.csv")
covid_df = covid_df[["Country_Region", "Confirmed", "Deaths", "Case_Fatality_Ratio"]]
grouped = covid_df.groupby("Country_Region").sum()

life_expectancy_df = pd.read_csv("life_expectancy_data.csv")
life_expectancy_df.columns = ["Country", "Life Expectancy"]
life_expectancy_df.set_index("Country", inplace=True)

# function that returns two global variables: 
# "random": which is a df of random countries that fit the fatality ratio filter conditions,
# "random_list": a list of the country names
# inputs: n is the number of countries, lower and upper are the lower and upper bounds of the fatality rate
def random_countries(n, lower, upper):
    grouped_filtered = grouped[(grouped["Case_Fatality_Ratio"] > lower) & (grouped["Case_Fatality_Ratio"] < upper)]
    global random 
    random = grouped_filtered.sort_values("Confirmed").sample(n=n)
    random.rename_axis("Country", inplace=True)
    global random_list
    random_list = list(random.index)
    
    
# function that returns two plots for "Deaths" and "Cases" Vs life expectancy for the randomly selected countries
def plot_life_expectancy(): 
    random_life_expectancy = life_expectancy_df[life_expectancy_df.index.isin(random_list)]
    covid_life_expectancy = pd.concat([random, random_life_expectancy], axis="columns", join="inner")
    return covid_life_expectancy.hvplot.scatter(x="Life Expectancy", y="Deaths", by="Country", title="Life Expectancy Vs Deaths") + covid_life_expectancy.hvplot.scatter(x="Life Expectancy", y="Confirmed", by="Country", title="Life Expectancy Vs Cases")

# function that returns two plots for "Deaths" and "Cases" Vs health expenditure per person for the randomly selected countries
def plot_health_expenditure():
    random_health_df = health_expenditure_df[health_expenditure_df["Country"].isin(random_list)]
    random_health_df.set_index("Country", inplace=True)
    combined_df = pd.concat([random, random_health_df], axis="columns", join="inner")
    return combined_df.hvplot.scatter(x="International Dollar Per Capita (PPP)", y="Deaths", by="Country", title="Health Expenditure Per Person Vs Deaths") + combined_df.hvplot.scatter(x="International Dollar Per Capita (PPP)", y="Confirmed", by="Country", title="Health Expenditure Per Person Vs Cases")

In [17]:
# create widgets
n_slider = pn.widgets.IntSlider(name='Set No. of Countries', start=10, end=100, step=5, value=30)
lower_slider = pn.widgets.FloatSlider(name='Set min fatality rate', start=0, end=5, step=.1, value= 0.8)       
upper_slider = pn.widgets.FloatSlider(name='Set max fatality rate', start=0, end=5, step=.1, value = 1.5)
button = pn.widgets.Button(name='Generate data and graph', button_type='primary')

# A helper function to call random_countries when button is clicked. Must be declared after the widgets have been created since it references them.
def random_countries_f(event):
    random_countries(n_slider.value, lower_slider.value, upper_slider.value)
    layout[1].object = plot_health_expenditure()
    layout[2].object = plot_life_expectancy()
    

# initiate starting data
random_countries(30, .8, 2.5)

button.on_click(random_countries_f)

layout = pn.Column(pn.Column(n_slider, lower_slider, upper_slider, button), plot_health_expenditure(), plot_life_expectancy())

layout.servable()

Traceback (most recent call last):
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\pyviz_comms\__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\panel\viewable.py", line 272, in _on_msg
 patch.apply_to_document(doc, comm.id)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in apply_to_document
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 1198, in _with_self_as_curdoc
 return f()
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in <lambda>
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 398, in apply_json_patch
 self._trigger_on_message(event_json["msg_type"], event_json["msg_data"])
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 687, in _trigger_on_message
 cb(msg_data)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 356, in apply_json_event
 model._trigger_event(event)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\util\callback_manager.py", line 91, in _trigger_event
 self._document._with_self_as_curdoc(invoke)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 1198, in _with_self_as_curdoc
 return f()
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\util\callback_manager.py", line 80, in invoke
 callback(event)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\panel\widgets\button.py", line 119, in _server_click
 self._change_event(doc)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\panel\reactive.py", line 288, in _change_event
 self._process_events(events)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\panel\reactive.py", line 262, in _process_events
 self.param.set_param(**self_events)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 1526, in set_param
 self_._batch_call_watchers()
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 1665, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\param\parameterized.py", line 1627, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-17-3b75ab0e763f>", line 9, in random_countries_f
 random_countries(n_slider.value, lower_slider.value, upper_slider.value)
 File "<ipython-input-16-4e796d4bd34c>", line 20, in random_countries
 random = grouped_filtered.sort_values("Confirmed").sample(n=n)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\generic.py", line 5350, in sample
 locs = rs.choice(axis_length, size=n, replace=replace, p=weights)
 File "mtrand.pyx", line 959, in numpy.random.mtrand.RandomState.choice
ValueError: Cannot take a larger sample than population when 'replace=False'
 


 Traceback (most recent call last):
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\pyviz_comms\__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\panel\viewable.py", line 272, in _on_msg
 patch.apply_to_document(doc, comm.id)
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in apply_to_document
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\chapm\anaconda3\envs\pyvizenv\lib\site-packages\bokeh\document\document.py", line 1198, in _with_self_as_curdoc
 return f()
 File "C:\Users\chapm\anaconda3\envs

Column
    [0] Column
        [0] IntSlider(end=100, name='Set No. of Countries', start=10, step=5, value=30)
        [1] FloatSlider(end=5, name='Set min fatality rate', value=0.8)
        [2] FloatSlider(end=5, name='Set max fatality rate', value=1.5)
        [3] Button(button_type='primary', name='Generate data and graph')
    [1] HoloViews(Layout)
    [2] HoloViews(Layout)